In [ ]:
!ipcluster start --n=4 --profile=mycluster1

In [ ]:
# figure out the start and kill cluster at the end
# threading for RAM usage

In [ ]:
def decorator_1(func):
    def func_wrapper(x):
        print('dec1')
        func(x)
        print('dec1')
    return func_wrapper

def decorator_2(func):
    def func_wrapper(x):
        print('dec2')
        func(x)
        print('dec2')
    return func_wrapper

@decorator_2
@decorator_1
def function_a(x):
    print(x)

function_a(1)

In [6]:
import os

os.system("ipcluster start --n=4 --profile=mycluster0 --daemonize")

0

In [ ]:
# engine 

In [ ]:
# each cluster will have to decorate the mapper/reducer?

In [ ]:
after each map/reducer step, use gc.collect()
# figure out how to send to larger cluster? It appears that's not as important.
# Previously, we sent large files to large cluster because of making sure RAM usage is fine.
# Now, the cluster will kill itself if RAM usage is exceeded. However, now loss of cluster.
# Hmm, ideally still sent large file to large cluster

In [ ]:
# RAM usage is heavier in Python 3 than Python 2; though Python 3 memory management is better

In [ ]:
# suppose we have equal sized clusters, how to distribute load?
# write about where to do it queue or have large/small cluster distinction

In [ ]:
# SSD for parallel reading (not HDD); determine if you are IO constrained

In [19]:
# MCE works on files. Hence, if you don't have any datafiles, then just create some empty files

In [11]:
# create some fake files
for i in range(10):
    !touch {i}.tmp

In [44]:
def temp_f(name1):
    print('hello {}'.format(name1))

def mapper_f(kwargs):
    return temp_f(**kwargs)

In [49]:
mapper_f({'name1': 'lee'})
(lambda kwargs: temp_f(**kwargs))({'name1': 'lee'})

hello lee
hello lee


In [51]:
import copy

temp_dict = copy.deepcopy({'a': 'b'})
temp_dict.update({'c': 'd'})
temp_dict

In [1]:
import time
import os
import ipyparallel as ipp
from collections import defaultdict
from tqdm import tqdm
import itertools
from datetime import datetime
        
class MultipleClusterEnginePrototype(object):
    def __init__(self, cluster_name, n_cpus_list, file_names, function_to_process, function_args_dict): # always put it in as a dictionary
        self.cluster_name = cluster_name
        self.n_cpus_list = n_cpus_list
        self.file_names = file_names
        self.function_to_process = lambda kwargs: function_to_process(**kwargs)
        self.function_args_dict = function_args_dict

        # used by engine
        self.client_dict = {}
        self.load_balanced_view_dict = {}
        self.async_results_dict = defaultdict(list) # collects all the async_results
        self.cluster_indexes = None
        
    def start_cluster(self, n_cpus, cluster_id):
        print('\tAttempting to start {}{} with {} CPUs'.format(self.cluster_name, cluster_id, n_cpus))
        os.system("ipcluster start --n={} --profile={}{} --daemonize".format(
            n_cpus, self.cluster_name, cluster_id)) # should deprecate to use a safer bash call

        attempt_ctr = 0 
        while attempt_ctr < 3: # Attempt to connect to client 3 times
            time.sleep(10) # hard coded
            try:
                client = ipp.Client(profile='{}{}'.format(self.cluster_name, cluster_id))
            except ipp.error.TimeoutError:
                attempt_ctr += 1
            else:
                print('\t\tCPU processes ready for action: {}'.format(client[:].apply_async(os.getpid).get()))
                return client
            # if there is any other error other than TimeoutError, then the error will be raised
    
    def start_all_clusters(self):
        print('\nAttempting to start all clusters')
        for cluster_id, n_cpus in enumerate(self.n_cpus_list):
            self.client_dict[cluster_id] = self.start_cluster(n_cpus, cluster_id)
            self.load_balanced_view_dict[cluster_id] = self.client_dict[cluster_id].load_balanced_view()            
        print('All clusters started\n')
        self.cluster_indexes = itertools.cycle(sorted(self.load_balanced_view_dict))
        
    def kill_cluster(self, cluster_id): # use better arguments
        # client = client_list[cluster_id]
        print('\tAttempting to kill {}{} with CPU processes: {}'.format(
            self.cluster_name, cluster_id, self.client_dict[cluster_id][:].apply_async(os.getpid).get()))
        self.load_balanced_view_dict.pop(cluster_id)
        # client.purge_everything()
        self.client_dict[cluster_id].close()
        os.system('ipcluster stop --profile={}{}'.format(self.cluster_name, cluster_id))
        print('\t\tCluster successfully killed')
        time.sleep(5) # hard-coded
        # have to mutate cluster_indexes
        
    def kill_all_clusters(self):
        print('\nAttempting to kill all clusters')
        for cluster_id in self.client_dict:
            self.kill_cluster(cluster_id)
        print('All clusters have been killed\n')
        
    def run_clusters(self):        
        small_file_ctr = 1 # determine if you want to have queue or differently ordered queue
        big_file_ctr = 0
        

        for ith_file in tqdm(range(len(self.file_names))):
            for jth_cluster in self.cluster_indexes: # infinite loop
                time.sleep(1) # hard coded delay time; want to do expected log time lag / number of clusters
                ### insert code here to kill cluster if RAM usage too great, if possible log which file it was processing;
                ### it has to do a global search of all clusters' RAM usage
                ### would need a dictionary here to remember which cluster has which file; write to disk
                ### profiler would also write to disk CPU usage what level

                if (not self.async_results_dict[jth_cluster][-1:] 
                    or self.async_results_dict[jth_cluster][-1].done()): # check if cluster i is available                       
                    # if necessary, recreate engine here
                    if jth_cluster == 0: # Send large files to large cluster (ALWAYS has id == 0)
                        index = big_file_ctr
                        big_file_ctr += 1
                    else: # Send small files to small clusters (ALWAYS have id > 0)
                        index = -small_file_ctr
                        small_file_ctr += 1
                                              
                                     
                    # clear cluster memory 

                    # package_arguments
                    args = {} + function_args_dict
                    
                    
                    ### insert code to write results to file--it will only have start times, no end times
                    async_result = self.load_balanced_view_dict[jth_cluster].map_async(
                            self.function_to_process, # function name
                            [self.file_names[index]] * len(self.client_dict[jth_cluster].ids), # file name, assumes first argument is always file name
                            # [len(client_list[i].ids)] * len(client_list[i].ids), # number of CPUs, assumes second argument is always number of CPUs
                            #  client_list[i].ids # CPU ids, assumes third argument is always CPU id; actually turn into kwargs
                             # [output_folder_name] * len(client_list[i].ids) # assumes fourth argument is output directory
                            [self.function_args_dict] * len(self.client_dict[jth_cluster].ids)
                            )
                                              
                    self.async_results_dict[jth_cluster].append(async_result)
                    print("{} is the {}th file and is sent to {}{} for processing".format(self.file_names[index], ith_file, self.cluster_name, jth_cluster))
                    break # break out of inner loop to determine if other clusters are available
        # async_results_dict; save to disk for later inspection?
        
    def main(self):
        start_time = datetime.now()
        print('\nStarting Multiple Cluster Engine at {}\n'.format(start_time))
        self.start_all_clusters()
        self.run_clusters()
        self.kill_all_clusters()
        end_time = datetime.now()
        print('\nMultiple Cluster Engine shut down at {}'.format(end_time))
        print('Total run time is {} minutes\n'.format((end_time - start_time).seconds / 60))

In [7]:
!cat 0.tmp

bobbypee-a-boo!

In [6]:
fun_func('0.tmp', **{'save_string_to_file': 'pee-a-boo!'})

In [2]:
def fun_func(file_name, save_string_to_file):
    with open(file_name, 'a') as f:
        f.write(save_string_to_file)

mce_args = {
    'cluster_name': 'mycluster',
    'n_cpus_list': [4, 3, 2], # 1st cluster is always the largest or equal to the other clusters
    'file_names': ['{}.tmp'.format(i) for i in range(10)],
    'function_to_process': fun_func,
    'function_args_dict': {'save_string_to_file': 'pee-a-boo!'}
    }

mce = MultipleClusterEnginePrototype(**mce_args)
#mce.start_all_clusters()
mce.main()


Starting Multiple Cluster Engine at 2018-02-24 01:00:03.373128


Attempting to start all clusters
	Attempting to start mycluster0 with 4 CPUs
		CPU processes ready for action: [8442, 8444, 8446, 8450]
	Attempting to start mycluster1 with 3 CPUs
		CPU processes ready for action: [8513, 8515, 8517]
	Attempting to start mycluster2 with 2 CPUs


  0%|          | 0/10 [00:00<?, ?it/s]

		CPU processes ready for action: [8573, 8575]
All clusters started



 10%|█         | 1/10 [00:01<00:09,  1.00s/it]

0.tmp is the 0th file and is sent to mycluster0 for processing


...

 90%|█████████ | 9/10 [00:09<00:01,  1.00s/it]

4.tmp is the 8th file and is sent to mycluster2 for processing


100%|██████████| 10/10 [00:10<00:00,  1.00s/it]

3.tmp is the 9th file and is sent to mycluster0 for processing

Attempting to kill all clusters
	Attempting to kill mycluster0 with CPU processes: [8442, 8444, 8446, 8450]


		Cluster successfully killed
	Attempting to kill mycluster1 with CPU processes: [8513, 8515, 8517]
		Cluster successfully killed
	Attempting to kill mycluster2 with CPU processes: [8573, 8575]
		Cluster successfully killed
All clusters have been killed


Multiple Cluster Engine shut down at 2018-02-24 01:01:01.541443
Total run time is 0.9666666666666667 minutes



In [ ]:
mce.run_clusters()

In [22]:
!cat 1.tmp

In [4]:
mce.async_results_dict

defaultdict(list,
            {0: [<AsyncMapResult: fun_func:finished>,
              <AsyncMapResult: fun_func:finished>,
              <AsyncMapResult: fun_func:finished>,
              <AsyncMapResult: fun_func:finished>],
             1: [<AsyncMapResult: fun_func:finished>,
              <AsyncMapResult: fun_func:finished>,
              <AsyncMapResult: fun_func:finished>],
             2: [<AsyncMapResult: fun_func:finished>,
              <AsyncMapResult: fun_func:finished>,
              <AsyncMapResult: fun_func:finished>]})

In [87]:
temp = mce.async_results_dict[0][-1]

In [14]:
mce.kill_all_clusters()
#!ipcluster stop --profile=mycluster2


Attempting to kill all clusters
All clusters have been killed



In [16]:
!ipcluster stop --profile=mycluster1

2018-02-24 00:59:17.380 [IPClusterStop] CRITICAL | Could not read pid file, cluster is probably not running.


In [ ]:
class MultipleClusterEngine(object):
    def __init__(self, **kwargs):
        self.functions_to_run = functions_to_run
        self.file_names = file_names
        self.RAM_limit_in_GB = pass
        # how to deal with other args/objects that functions might need
        self.mce_job_name = None
        pass
   

    def memory_profiler():
        pass # if all clusters are dead, then raise Error with a message
    
    
    def early_kill():
        pass # write file to failure disk, maybe also cluster i and num_cpus
            
    def running_cluster():
        # Create asynchronous object and cycle
        my_async_results_list = defaultdict(list)
        
        small_file_ctr = 1 # determine if you want to have queue or differently ordered queue
        big_file_ctr = 0

        for ith_file in tqdm(range(len(file_names))):
            for jth_cluster in itertools.cycle(range(len(load_balanced_view_list))): # infinite loop
                time.sleep(1) # hard coded delay time; want to do expected log time lag / number of clusters
                ### insert code here to kill cluster if RAM usage too great, if possible log which file it was processing;
                ### it has to do a global search of all clusters' RAM usage
                ### would need a dictionary here to remember which cluster has which file; write to disk
                ### profiler would also write to disk CPU usage what level

                if (not my_async_results_list[jth_cluster][-1:] or my_async_results_list[jth_cluster][-1].done()): # check if cluster i is available                       
                    # if necessary, recreate engine here
                    if jth_cluster == 0: # Send large files to large cluster (ALWAYS has id == 0)
                        index = big_file_ctr
                        big_file_ctr += 1
                    else: # Send small files to small clusters (ALWAYS have id > 0)
                        index = -small_file_ctr
                        small_file_ctr += 1
                                              
                                     
                    # clear cluster memory 

                    ### insert code to write results to file--it will only have start times, no end times
                    async_object = load_balanced_view_list[jth_cluster].map_async(
                            mapper_reducer_function, # function name
                            [file_names[index]] * len(client_list[jth_cluster].ids), # file name, assumes first argument is always file name
                            # [len(client_list[i].ids)] * len(client_list[i].ids), # number of CPUs, assumes second argument is always number of CPUs
                            #  client_list[i].ids # CPU ids, assumes third argument is always CPU id; actually turn into kwargs
                             # [output_folder_name] * len(client_list[i].ids) # assumes fourth argument is output directory
                            [function_args_dict] * len(client_list[jth_cluster].ids)
                            )
                                              
                    my_async_results_list[jth_cluster].append(async_object)
                    print("{} is the {}th file and is sent to cluster {} for processing".format(file_names[index], ith_file, i))
                    break # break out of inner loop to determine if other clusters are available
        # my_async_results_list; save to disk for later inspection?
                            
    def main():
        start_time = now()
        self.spool_engines()
        self.running_clusters()
        self.shutdown_clusters()
        print('Total Run Time is {}'.format(now() - start_time))

In [ ]:
; if cluster is killed, then cannot trust jth_cluster index--instead of load_balanced_list, use load_balanced_dict

In [ ]:
# RAM writer and also progress writer and failre writer
# figure out queue vs deque; deque is better

In [ ]:
# Choose appropriate files folder, based on emitter type
file_names = ['invoices0.txt', 'invoices1.txt']
num_files = len(file_names)
print 'There are ' + str(num_files) + ' files to process'
assert num_files > 0, 'File list empty'

# Define global computing parameters
# There is one big cluster for 50gb+ files (if present) and two clusters for smaller files.
cpu_list = [1,1]
if len(arr0):
    cpu_list = arr0
num_clusters = len(cpu_list)
client_list = []
load_balanced_view_list = []

# Start your engines!
for i in range(num_clusters):
    client_list.append(start_cluster(i, cpu_list[i]))
    load_balanced_view_list.append(client_list[i].load_balanced_view())

# Create asynchronous object and cycle
my_async_results_list = defaultdict(list)
cluster_index = itertools.cycle(range(len(load_balanced_view_list)))

# Progress check!
for i in range(num_clusters):
    print 'Client ids for cluster ' + str(i) + ': ' + str(client_list[i].ids)
print str(num_clusters) + ' clusters activated!'
print '\n Client connection checkpoint reached!'

# Record total processing time
now = datetime.today()

# Distribute small and large files to small and large clusters
small_file_ctr = 1
big_file_ctr = 0
for file_idx in range(len(file_names)):
    for i in cluster_index: # infinite loop
        time.sleep(1) # hard coded delay time
                    
        if (not my_async_results_list[i][-1:] or my_async_results_list[i][-1].done()): # check if cluster i is available           
            num_cpus = len(client_list[i].ids)
        
            # Must de-bug cluster shutdown code
            kill_cluster(i, client_list)
                
            #For de-bugging
            print 'stopped cluster '+ str(i)
            client_list[i] = start_cluster(i, num_cpus)
            load_balanced_view_list[i] = client_list[i].load_balanced_view()
            
            # Send large files to large cluster (ALWAYS has id == 0)
            if i == 0: 
                index = big_file_ctr
                big_file_ctr += 1
            # Send small files to small clusters (ALWAYS have id > 0)
            else:    
                index = -small_file_ctr
                small_file_ctr += 1
                
            print "file is {}, {}".format(file_names[index], index)
            temp = load_balanced_view_list[i].map_async(
                    process_file, # function name
                    [file_names[index]] * len(client_list[i].ids), # file name
                    [len(client_list[i].ids)] * len(client_list[i].ids), # number of CPUs
                    client_list[i].ids # CPU ids
                    ) 
            my_async_results_list[i].append(temp)
            print "File is sent to cluster {}".format(i) 
            break
        
# kill all existing clusters
for i in range(len(client_list)):
    kill_cluster(i, client_list)        

small_file_ctr -= 1
with open('time_stamp.txt', 'a') as f:
    f.write(str(datetime.today()) + '\n')
    to_write = 'Processed ' + str(num_files) + ' files in ' \
            + str((datetime.today() - now).total_seconds() / 60) + ' minutes\n'
    to_write += 'Used ' + str(cpu_list[0]) + ' cpus to process ' + str(big_file_ctr) + \
                    ' big files and ' 
    # If there are multiple clusters, specify number of small files processed by small cluster
    if len(cpu_list) > 1: 
        to_write += str(cpu_list[1]) + ' cpus to process ' + \
                    str(small_file_ctr) + ' small files.\n'
    to_write += 'A total of ' + str(len(cpu_list)) + ' clusters were used.\n'
    if len(cpu_list) > 1: 
        to_write += 'Expect ' + str(big_file_ctr * cpu_list[0] + small_file_ctr * cpu_list[1]) + \
                    ' files in processed folder.\n\n'
    else:
        to_write += 'Expect ' + str(big_file_ctr * cpu_list[0]) + \
                    ' files in processed folder.\n\n'
    f.write(to_write)
    
print 'Done processing!'